In [1]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library


CondaValueError: too few arguments, must supply command line package specs or --file



<b> Installing Beautifulsoup4</b>

In [4]:
!conda install -c anaconda beautifulsoup4

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    beautifulsoup4-4.9.0       |           py36_0         165 KB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.9.0-py36_0      anaconda
    ca-certificates: 2020.1.1-0        --> 2020.1.1-0        anaconda
    certifi:         2020.4.5.1-py36_0 --> 2020.4.5.1-py36_0 anaconda
    openssl:         1.1.1f-

Need to get data from this address: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [50]:
from bs4 import BeautifulSoup

url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

import requests
response = requests.get(url)
#print (response.status_code) #Should be 200 for a good request
#print (response.content)

soup = BeautifulSoup(response.content, 'html.parser')


OK now we have soup! Let's pase what we got

In [75]:
gdp_table = soup.find("table", attrs={"class": "wikitable"})
gdp_table_data = gdp_table.tbody.find_all("tr") 
print("Number of rows = " + str(len(gdp_table_data)))

#let's get the headers
headings = []
for th in gdp_table_data[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(th.string.replace('\n', ' ').strip())

print("The Table Headers are:")
print(headings)



Number of rows = 181
The Table Headers are:
['Postal code', 'Borough', 'Neighborhood']


In [111]:
df_q1 = pd.DataFrame(columns=headings)
contents = []

for row in gdp_table_data[1:100]:
    myrow = []
    for td in row.find_all("td"):
        # remove any newlines and extra spaces from left and right
        myrow.append(td.string.replace('\n', ' ').strip())
    #If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
    if myrow[2] == "":
        myrow[2] = myrow[1]
    df_q1.loc[len(df_q1)] = myrow
    
df_q1["Borough"].replace("Not assigned", np.nan, inplace=True)
df_q1.dropna(subset=["Borough"], inplace=True)
df_q1.reset_index(drop=True, inplace=True)
print("Number of rows = ", str(len(df_q1)))
df_q1


Number of rows =  65


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma


In [109]:
#Show the issue the question is talking about:
df_q1.loc[df_q1['Postal code'] == "M5A"]

,Postal code,Borough,Neighborhood
2,M5A,Downtown Toronto,Regent Park / Harbourfront


Hmmm, looks like they are already combined in the raw dataset. We can replace the "/" with a ",". Seems pointless, So here is code that would combine text from same rows with a ",", if they existed in seperate rows.

In [115]:
df_q1 = df_q1.groupby(['Postal code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_q1.tail()

,Postal code,Borough,Neighborhood
60,M9B,Etobicoke,West Deane Park / Princess Gardens / Martin Gr...
61,M9C,Etobicoke,Eringate / Bloordale Gardens / Old Burnhamthor...
62,M9L,North York,Humber Summit
63,M9M,North York,Humberlea / Emery
64,M9N,York,Weston


In [116]:
print("As you can see the number of rows is still " + str(len(df_q1)))

As you can see the number of rows is still 65


So we ansered the question of combining like zip code neighborhoods, but in this dataset it is unneccesary.

Finnaly, they want us to do this: 
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [118]:
df_q1.shape

(65, 3)